In [2]:
import math
import sys
import pickle
from math import log10 as log

import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

from pandas.plotting import scatter_matrix
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, f1_score, roc_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder


In [3]:
# Load data
df_train_in = pd.read_csv('X_train.txt', header=None, delim_whitespace=True)
df_train_out = pd.read_csv('y_train.txt', header=None, delim_whitespace=True)

df_test_in = pd.read_csv('X_test.txt', header=None, delim_whitespace=True)
df_test_out = pd.read_csv('y_test.txt', header=None, delim_whitespace=True)

In [4]:
# Pre processing
'''
OneHotEncode    0–caminhada; 1 –subindoescadas; 2 –descendo escadas; 3 –sentado; 4 –em pé; 5 –deitado.
'''
encoder = OneHotEncoder(categories='auto')
encoder.fit(df_train_out)

df_train_out_scaled = encoder.transform(df_train_out).toarray()
df_test_out_scaled = encoder.transform(df_test_out).toarray()

# Normalization
scaler = StandardScaler() 
scaler.fit(df_train_in)

df_train_in_scaled = scaler.transform(df_train_in)
df_test_in_scaled = scaler.transform(df_test_in)

In [5]:
def get_distance(inp, train, p):
    return np.argsort(np.sum((train - inp)**p, axis=1)**(1/p))

def make_prediction(k, distances):
    votes = [df_train_out.to_numpy()[e] for e in distances[0:k]]
    unique, counts = np.unique(votes, return_counts=True)
    pred = dict(zip(unique, counts))
    return max(pred, key=lambda key: pred[key])
 

In [6]:
# Calc Distance
p = 2
distances_list = []
idx = 0
for test_in in df_test_in_scaled:
    distances_list.append(get_distance(test_in, df_train_in_scaled, p))
    sys.stdout.write("\r{}".format(idx))
    sys.stdout.flush()
    idx += 1

2946

In [8]:
# Predict
k = 1000
k = k if k<len(distances_list[0]) else len(distances_list[0])
print('k',k)
test_out_predict = []
for distances in distances_list:
    test_out_predict.append(make_prediction(k, distances))

cm = confusion_matrix(df_test_out.to_numpy(),test_out_predict)
print('Test')
print(cm)


k 1000
Test
[[492   3   1   0   0   0]
 [ 72 396   3   0   0   0]
 [177  55 188   0   0   0]
 [  0   4   0 178 309   0]
 [  0   1   0   7 524   0]
 [  0   1   0  10  41 485]]
